In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import *
from sklearn.preprocessing import *
from sklearn.model_selection import *

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10402,10402,128.0,7.558488,12.000000,4.000000,26.385218,11.330440,2.644000,1.631000,0.892000,0.496070,1.79607,4.0
10403,10403,30.0,1.743160,10.000000,5.333333,20.766935,14.163933,3.090000,1.556667,0.866667,0.480390,0.81480,5.0
10404,10404,196.0,30.920000,24.500000,5.500000,53.490297,10.074300,2.295000,1.545000,1.120000,0.469715,2.11540,1.8
10405,10405,38.0,1.553160,12.666667,4.666667,26.621687,11.290033,2.743333,1.756667,0.980000,0.486507,0.77755,6.0


In [4]:
train_input = train.drop(columns=["id","Hardness"])
train_target = train["Hardness"]
test_input = test.drop(columns=["id"])

In [5]:
f,p = f_regression(train_input,train_target)
p_values = pd.DataFrame(columns=["feature","p-value"])
p_values["feature"] = train_input.columns
p_values["p-value"] = p.round(2)
p_values

,feature,p-value
0,allelectrons_Total,0.0
1,density_Total,0.0
2,allelectrons_Average,0.0
3,val_e_Average,0.0
4,atomicweight_Average,0.0
5,ionenergy_Average,0.0
6,el_neg_chi_Average,0.0
7,R_vdw_element_Average,0.0
8,R_cov_element_Average,0.0
9,zaratio_Average,0.0


In [6]:
ss = StandardScaler()

train_input_1 = ss.fit_transform(train_input)
train_target_1 = train_target.copy()

test_input_1 = ss.fit_transform(test_input)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(train_input_1,train_target_1,test_size=0.3,random_state=np.random.randint(0,1000))

X_test = test_input_1.copy()

In [12]:
element_size = X_train[0].shape

In [13]:
element_size

(11,)

In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=16,activation="relu",input_shape=element_size),
    tf.keras.layers.Dense(units=32,activation="relu"),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="linear")
])

In [16]:
model.compile(
    optimizer = tf.keras.optimizers.legacy.Adam(),
    loss = tf.keras.losses.mean_squared_error,
    metrics = tf.keras.metrics.mean_squared_error
)

In [17]:
model.fit(
    x=X_train, y=y_train,
    batch_size = 32,
    epochs = 100,
    verbose = 1,
    callbacks = tf.keras.callbacks.EarlyStopping(),
    validation_data=(X_val,y_val),
    shuffle = True,
    validation_batch_size = 32
)

Epoch 1/100
228/228 [==============================] - 0s 784us/step - loss: 5.8500 - mean_squared_error: 5.8500 - val_loss: 2.7651 - val_mean_squared_error: 2.7651
Epoch 2/100
228/228 [==============================] - 0s 523us/step - loss: 2.1395 - mean_squared_error: 2.1395 - val_loss: 2.0969 - val_mean_squared_error: 2.0969
Epoch 3/100
228/228 [==============================] - 0s 538us/step - loss: 1.9165 - mean_squared_error: 1.9165 - val_loss: 1.9717 - val_mean_squared_error: 1.9717
Epoch 4/100
228/228 [==============================] - 0s 547us/step - loss: 1.8310 - mean_squared_error: 1.8310 - val_loss: 1.8287 - val_mean_squared_error: 1.8287
Epoch 5/100
228/228 [==============================] - 0s 524us/step - loss: 1.8055 - mean_squared_error: 1.8055 - val_loss: 1.9605 - val_mean_squared_error: 1.9605


In [20]:
mse, mse_2 = model.evaluate(X_test)

217/217 [==============================] - 0s 251us/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00


In [27]:
df_final = pd.DataFrame(columns=["id","Hardness"])

df_final["id"] = test["id"].copy()
df_final["Hardness"] = model.predict(X_test).round(3)

217/217 [==============================] - 0s 245us/step


In [28]:
df_final

,id,Hardness
0,10407,3.557
1,10408,4.254
2,10409,6.065
3,10410,4.745
4,10411,5.857
...,...,...
6934,17341,5.717
6935,17342,4.357
6936,17343,6.297
6937,17344,3.661


In [29]:
df_final.to_csv("amith_submission.csv",index=False,header=True)